In [1]:
using SummationByPartsOperators
using Plots
using Revise
using Distributions
using FFTW
using LinearAlgebra
using Base.Threads
using HDF5
using FileIO
using JLD2
 
includet("../PIC/PIC-1D/aux_functions/aux_functions.jl")
#includet("../PIC-1D/aux_functions/aux_functions.jl")
includet("local_aux_functions.jl")

In [ ]:

#data_name = "data_3d_h_step_cut_small_bola_F_full_T200.jld2"
#data_name = "data_3d_h_step_cut_medium_b_F_full_T200.jld2"
data_name = "data_3d_h_step_cut_small_bola_F_full_T200.jld2_F_full_T0.0_SF4em3.jld2"
@show data = load("Results/" * data_name)

The medium with source_factor = 0.004^2 failed to complete with F_full. 
The small with same source_factor (I think did evolved).

In [26]:

par_grid = data["par_grid"]
Box_x, J, n_fields = par_grid
par_evolv  = data["par_evolv"]
T, data_steps, int_steps, dt = par_evolv
run_name = data["run_name"]
size(data["u/u_00002"])
v = zeros(data_steps, size(data["u/u_00002"])...)
m = 3 #data_steps - 1
for m in 1:m
    tiempo = @sprintf("%05d", m)
    v[m,:,:,:,:] = data["u/u_$(tiempo)"]
end

In [ ]:
#surface(v[3,1,:,:,J[3] ÷ 2].-1 )
heatmap(v[m,1,:,:,J[3] ÷ 2].-1, aspectratio=1 )

In [ ]:
plot(v[m,3,:,J[2] ÷ 2 , J[3]÷ 2]./v[m,1,:,J[2] ÷ 2 , J[3]÷ 2] .- 1, label="lapse")
plot!(v[m,1,:,J[2] ÷ 2 , J[3]÷ 2].- 1, label="factor conforme - 1")
#png("lapse and conformal factor after half crossing time")



In [ ]:
#m = 9
plot([get_norm_time(v,i)[1].-1 for i in 1:m], label="ϕ - 1" )
plot!([get_norm_time(v,i)[2] for i in 1:m], label="ϕ_t" )
plot!([get_norm_time(v,i)[3].-1 for i in 1:m], label="v - 1")
plot!([get_norm_time(v,i)[4] for i in 1:m], label="v_t")